**Connect with google drive**

In [67]:
!pip install yfinance 

In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Step_1: Get the data of 20 companies from yahoo web and store in google drive**

In [69]:
from pandas_datareader import data as pdr
from datetime import date
import yfinance as yf
yf.pdr_override()
import pandas as pd
#datetime is a Python module
import datetime

# Set the start and end date
start_date = '2017-01-01'

#start_date = '2017-01-01'
# Get data to today
today = datetime.datetime.today()
# Set the list including 20'str' acronym of 20 companies
list = ['INTC','AMD', 'CSCO', 'AAPL', 'MU', 'NVDA', 'QCOM', 'AMZN', 'NFLX', 'FB', 'GOOG', 'BABA', 'EBAY', 'IBM', 'XLNX', 'TXN', 'NOK', 'TSLA', 'MSFT', 'SNPS']

# Get the data
files=[]

# Create a data folder in my google drive.
def SaveData(df, filename):
  df.to_csv('/content/drive/MyDrive/BAP-AI/LSTM/Dataset/'+filename+'.csv')

def GetData(acronym):
  print (acronym)
  data = pdr.get_data_yahoo(acronym, start = start_date, end = today )
  dataname= acronym
  files.append(dataname)
  # SaveData(data, dataname)

print("Today is " + str(today))
#Get data, and save that data as "str".csv
for i in list:
  GetData(i)
  

Today is 2021-07-24 17:12:06.366814
INTC
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
CSCO
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
MU
[*********************100%***********************]  1 of 1 completed
NVDA
[*********************100%***********************]  1 of 1 completed
QCOM
[*********************100%***********************]  1 of 1 completed
AMZN
[*********************100%***********************]  1 of 1 completed
NFLX
[*********************100%***********************]  1 of 1 completed
FB
[*********************100%***********************]  1 of 1 completed
GOOG
[*********************100%***********************]  1 of 1 completed
BABA
[*********************100%***********************]  1 of 1 completed
EBAY
[*********************100%***********************]  1 of 1 completed
IBM
[**

### Requirement: 
30 ngày liên tiếp là input đầu vào của cột Adj => Predict ra cho ngày 31 

**Step_2: Working with LSTM model**

In [70]:
import tensorflow as tf 
from tensorflow.keras.preprocessing import sequence
import numpy as np


In [71]:
from sklearn.preprocessing import MinMaxScaler
data = pd.read_csv('/content/drive/MyDrive/BAP-AI/LSTM/Dataset/FB.csv')
data = data.filter(['Adj Close']).values
# data_train = np.array(data_train)
train_length = len(data) // 10 * 8

# Scale data to [0;1]
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data)
scaled_data

array([[0.        ],
       [0.00766397],
       [0.0159561 ],
       ...,
       [0.92172711],
       [0.94145234],
       [0.96059133]])

In [72]:
# Create x_train and y_train
x_train = []
y_train = []

for i in range(30, train_length):
  x_train.append(scaled_data[i-30:i,0])
  y_train.append(scaled_data[i, 0])

# Convert the x_train and y_train to numpy arrays 
x_train, y_train = np.array(x_train), np.array(y_train)
# Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

(882, 30, 1)

In [101]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Model
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(32))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='Adamax', loss='mean_squared_error')

# Train model
model.fit(x_train, y_train, batch_size=4, epochs=2)

Epoch 1/2
221/221 [==============================] - 29s 114ms/step - loss: 0.0099
Epoch 2/2
221/221 [==============================] - 25s 115ms/step - loss: 0.0013


**Step_3: Get results**

In [82]:
# Create data to test 
x_test = []
y_test = []

test_length = len(data) - train_length
for i in range(train_length, len(data)):
  x_test.append(scaled_data[i-30:i, 0])
  y_test.append(scaled_data[i, 0])

# Convert the x_test and y_test to numpy arrays 
x_test, y_test = np.array(x_test), np.array(y_test)
# Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_test.shape

(233, 30, 1)

In [83]:
# Get prediction of test data
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
# print(predictions)

**Step_4: Check**

In [102]:
index = train_length + 21
x = scaled_data[index:index + 30, 0]
x = np.reshape(x, (1, 30, 1))
prediction = model.predict(x)
prediction = scaler.inverse_transform(prediction)
print(prediction[0][0])
print(data[index + 30][0])



267.9156
280.82998657226557
